# ***Video da apresentação:***

---


# https://youtu.be/-5xjHpiqnL0 **bold text**

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
!pip install icc_rt
#!pip uninstall icc_rt

  Using cached https://files.pythonhosted.org/packages/1e/29/0ee89498c2b3e96620add5f4730d4e55e685b292a47aa67f52975fed9f71/icc_rt-2019.0-py2.py3-none-manylinux1_x86_64.whl


In [0]:
import pandas as pd
import numpy as np
import gensim 
import multiprocessing
import sklearn.preprocessing as pp
import warnings
import logging  # Setting up the loggings to monitor gensim
import numba
from IPython.display import display
from IPython.core.display import HTML
from gensim.models import Word2Vec,KeyedVectors 
from time import time
from gensim.models.phrases import Phrases, Phraser
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings('ignore')

#logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [0]:
t = time()
estaticos_market = pd.read_csv('/gdrive/My Drive/estaticos_market.csv')
#estaticos_market = pd.read_csv('/gdrive/My Drive/estaticos_portfolio1.csv')
print(estaticos_market.shape)
print('Time to Read Csv: {} mins'.format(round((time() - t) / 60, 2)))


(462298, 182)
Time to Read Csv: 0.17 mins


In [0]:
percent_missing = estaticos_market.isnull().sum() * 100 / len(estaticos_market)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing,'dtypes': estaticos_market.dtypes,})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df.head(181)

In [0]:
@numba.jit()
def fast_clean(df):
  t = time()
  col_exclude = ["fl_email", "fl_telefone", "qt_filiais", "tx_rotatividade", "tx_crescimento_24meses", "tx_crescimento_12meses", "qt_funcionarios_12meses", 
              "qt_funcionarios_24meses", "min_meses_servicos",'Unnamed: 0','fl_matriz','qt_alteracao_socio_total','qt_alteracao_socio_365d','qt_alteracao_socio_90d','grau_instrucao_macro_desconhecido']
  df = df.drop(col_exclude, axis=1)
  booleandf = df.select_dtypes(include=[bool]).columns
  booleanDictionary = {True: 'T', False: 'F'}
  for column in booleandf:
    df[column] = df[column].map(booleanDictionary)
  df= df.astype(str)
  objectdf  = df.select_dtypes(include=[object]).columns
  for column in objectdf:
    df[column] = df[column].str.replace(' ', '')
    df[column] = np.where(df[column] == 'nan', str(df.index), df[column])
    df[column] = column.replace('_', '')+"_"+df[column]
    df[column] = df[column].str.replace(',', '')
  Maker_Model = list(df.id.unique()) 
  indice = pd.Series(df.index, index=df['id']).drop_duplicates()
  df_id = pd.DataFrame({'id': df['id']})
  df = df.drop(['id'],axis=1)
  df['id'] = df_id['id']
  df2 = df_join(df) 
  df_clean = pd.DataFrame({'clean': df2})
  sent = [row.split(',') for row in  df_clean['clean']]
  print('Time to clean: {} mins'.format(round((time() - t) / 60, 2)))
  return(sent,indice,Maker_Model,df_clean)
def df_join(df):
  df2 = df.apply(lambda x: ','.join(x.astype(str)), axis=1)
  return(df2)

In [0]:
sent,indice, Maker_Model,df_clean = fast_clean(estaticos_market)

Time to clean: 2.63 mins


In [0]:
""" - UM DOS TESTE REALIZADOS - NÃO TEVE UM BOM DESEMPENHO
t = time()
tfidf = TfidfVectorizer(sublinear_tf=False,stop_words=None)
tfidf_matrix = tfidf.fit_transform(df_clean['clean'])
print('Time to tfidf: {} mins'.format(round((time() - t) / 60, 2)))
x = (tfidf_matrix.getrow(554).toarray().flatten())
y = (tfidf_matrix.getrow(9).toarray().flatten())
#x = np.squeeze(np.asarray(x))
#y = np.squeeze(np.asarray(y))
result = fast_cosine(x, y)
print(result)
tfidf_matrix = pp.normalize(tfidf_matrix.tocsc(), axis=0)
tfidf_matrix.shape
from scipy import sparse
b = (tfidf_matrix[9].transpose().todense())
b = np.squeeze(np.array(b), axis=1) 
len(b)
"""

In [0]:
t = time()
cores=5
model = Word2Vec(sent,min_count=1,size=300,workers=cores, window=3, sg = 0)
print('Time to build model: {} mins'.format(round((time() - t) / 60, 2)))

Time to build model: 4.68 mins


In [0]:
t = time()
model.wv.save_word2vec_format('/gdrive/My Drive/model.bin', binary=True)
print('Time to save: {} mins'.format(round((time() - t) / 60, 2)))

Time to save: 0.19 mins


In [0]:
t = time()
model2 = KeyedVectors.load_word2vec_format('/gdrive/My Drive/model.bin', binary=True)
#print('Time to load: {} mins'.format(round((time() - t) / 60, 2)))

In [0]:
def cosine_distance (model, word,target_list,num) :
    cosine_dict ={}
    try:
      a = model[word]
      for item in target_list :
          b = model[item]
          type(b)
          cos_sim = fast_cosine(a, b)
          cosine_dict[item] = cos_sim
      dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True) ## in Descedning order 
      data_day_list = []
      count = 1
      for item in dist_sort:
          data_day =(estaticos_market[estaticos_market.id==item[0][3:]])
          data_day_list.append(data_day)
          if count==num:
              break
          count=count+1
      final_data_day = pd.concat(data_day_list)
    except KeyError:
      print("")
    return final_data_day
@numba.jit()
def cosine_distance2 (tfidf_matrix,word,indice,target_list,num) :
    cosine_dict ={}
    word_list = []
    idx = indice[word]
    a = (tfidf_matrix.getrow(554).toarray().flatten())
    for item in target_list :
        if item != word :
            idx = indice[item]
            b = (tfidf_matrix.getrow(idx).toarray().flatten())
            b = np.squeeze(np.array(b))  
            cos_sim = fast_cosine(a, b)
            cosine_dict[item] = cos_sim
    dict_sorted(cosine_dict)
    data_day_list = []
    count = 1
    for item in dist_sort:
        print(item)
        data_day =(estaticos_market[estaticos_market.id==item[0][3:]])
        data_day_list.append(data_day)
        if count==num:
            break
        count=count+1
    final_data_day = pd.concat(data_day_list)
    return final_data_day
def dict_sorted(cosine_dict):
     dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True) ## in Descedning order 
     return(dist_sort)
@numba.jit(('f8,f8'),parallel=True,fastmath=True)
def fast_cosine(a, b):
    result  = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return(result)
@numba.jit(parallel=True,fastmath=False)
def requerimento(element,quant):
  t = time()
  result_list=[]
  element
  result = cosine_distance (model,element,Maker_Model,quant);
  print('Tempo de cálculo {} mins'.format(round((time() - t) / 60, 2)))
  return(result)

In [0]:
'''BLOCO DE TESTE
#id='id_dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9'
id='id_7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720'
#id='id_fc6969bffd9f104e3a148ad7df64b338ca885dd6a5aa5153b4754bd55746d638'
#a = model[id]
#model2.most_similar([id])
#cosine_distance (model,id,Maker_Model,10)
cosine_distance2 (tfidf_matrix,id,indice,Maker_Model,10)
#result = model2.similar_by_vector('id_'+x[0], topn= 50)
#data_day_list=[]
'''
tfidf = TfidfVectorizer(sublinear_tf=False,stop_words=None)
tfidf_matrix = tfidf.fit_transform(df_clean['clean'])
cosine_distance2 (tfidf_matrix,id,indice,Maker_Model,10)

In [0]:
estaticos_portfolio1 = pd.read_csv('/gdrive/My Drive/estaticos_portfolio1.csv')
estaticos_portfolio2 = pd.read_csv('/gdrive/My Drive/estaticos_portfolio2.csv')
estaticos_portfolio3 = pd.read_csv('/gdrive/My Drive/estaticos_portfolio3.csv')
print(estaticos_portfolio1.shape)
print(estaticos_portfolio2.shape)
print(estaticos_portfolio3.shape)
requis=(['7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720','70485e6b8abe52d8fb4e0bf060fb6f0f4f8576cb7583d885a8dcac38f506389','6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570'])
#requis=estaticos_portfolio3['id']
result_list=[]
result_list2=[]
for element in requis:
  try:
    id = 'id_'+element
    print(id)
    model[id]
    result=(requerimento(id,50))
    result_list.append(result)
  except Exception as e:
     print('chave não encontrada')
result_final = pd.concat(result_list)
result_final.drop_duplicates(keep = 'first', inplace = True)
result_list2 = []
for element in requis:
    result = (result_final[result_final.id==element]) 
    result_list2.append(result)
result_final2 = pd.concat(result_list2)
i1 = len(requis)
f1,f2 = result_final2.shape
print("Percentual de item solicitado no resultado: "+str(round(f1/i1*100,2))+'%')
pd.options.display.max_columns = None
display(result_final)

(555, 182)
(566, 2)
(265, 2)
id_7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720
Tempo de cálculo 0.35 mins
id_70485e6b8abe52d8fb4e0bf060fb6f0f4f8576cb7583d885a8dcac38f506389
chave não encontrada
id_6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570
Tempo de cálculo 0.35 mins
Percentual de item solicitado no resultado: 66.67%


,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
9946,9946,7d58990ba786a3a26617a8afbf814a9919ab34aa09013a...,True,SOCIEDADE EMPRESARIA LIMITADA,PI,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,5.309589,5 a 10,False,False,False,